The way to deal with cuboids that span with large arrays, the best is to consider their coordinates only

When adding cuboid (on), check against the existing cubes

If new cuboid overlap, then the new cuboid should be split into new cubes that do not overalap with the existing ones


The procedure to switch off needs also to be checked against the existing cuboid

If off region overlaps with existing cuboid then the cuboid will be split, generally into may non-overlapping cuboids, but it can simply be sliced, depending how it is done.

In [1]:
def checkOverlap(c1, c2):
    #Check second cuboid overlaps with first
    if c2[1][0]>c1[0][0] and c2[1][1]>c1[0][1] and c2[1][2]>c1[0][2] and \
        c2[0][0]<c1[1][0] and c2[0][1]<c1[1][1] and c2[0][2]<c1[1][2]:
        #print("overlaps")
        return True
    else:
        #print("no overlap")
        return False

In [2]:
c1 = [(0,0,0),(5,5,5)] # [ origin (x,y,z), end (x,y,z)] 
c2 = [(-2,-2,-2) , (0,5,5)]
checkOverlap(c1,c2)

False

In [3]:
def appendCube(c1, c2, subtract = False):
    #Check if overlap
    if checkOverlap(c1,c2):
        x1i, y1i, z1i = c1[0]
        x1f, y1f, z1f = c1[1]
        x2i, y2i, z2i = c2[0]
        x2f, y2f, z2f = c2[1]

        #print(f"x1i, y1i, z1i = {x1i}, {y1i}, {z1i}")
        #print(f"x1f, y1f, z1f = {x1f}, {y1f}, {z1f}")
        #print(f"x2i, y2i, z2i = {x2i}, {y2i}, {z2i}")
        #print(f"x2f, y2f, z2f = {x2f}, {y2f}, {z2f}")

        #Create a list of intervals for each axis

        def createIntervals(i1,f1 , i2,f2):
            #Returns a list of intervals and to which cuboid(s) it belongs to
            #  i1             f1
            #  ****************
            #          ----------
            #          i2       f2

            # ****************
            #       ----

            #    ***********
            # ------------------

            #    ***********
            # --------

            #print(f"createIntervals {i1},{f1} , {i2},{f2}")
            nextstart=None
            intervals=[]
            if i1<=i2:
                intervals.append( [i1,i2, [True, False]] )
                nextstart = i2
            else:
                intervals.append([i2,i1, [False, True]])
                nextstart=i1
            
            if f1<=f2:
                intervals.append([nextstart, f1, [True,True]])
                intervals.append([f1,f2, [False, True]])
            else:
                intervals.append([nextstart, f2,[True,True]])
                intervals.append([f2,f1, [True, False]])
            return intervals
        
        intervals_x = createIntervals(x1i, x1f, x2i, x2f)
        #print(f"intervals_x = {intervals_x}")
        intervals_y = createIntervals(y1i, y1f, y2i, y2f)
        #print(f"intervals_y = {intervals_y}")
        intervals_z = createIntervals(z1i, z1f, z2i, z2f)
        #print(f"intervals_z = {intervals_z}")

        #print("Process intervals")
        newCuboids = []
        #Create new cubes depending on the intervals
        for iz in intervals_z:
            #print(f"iz={iz}")
            for iy in intervals_y:
                #print(f"iy={iy}")
                for ix in intervals_x:
                    #print(f"ix={ix}")
                    #Check if interval belongs to the first cuboid in all axis
                    first = ix[2][0] and iy[2][0] and iz[2][0]
                    #print(f"first:{first}")

                    #Check if interval is in second cuboid in all axis
                    second = ix[2][1] and iy[2][1] and iz[2][1]
                    #print(f"second:{second}")

                    createCube = False
                    if not subtract:
                        #Add cuboids, or operation
                        createCube = first or second
                    else:
                        #Subtract operation, only retain first if not the second cuboid in this volume
                        createCube = first and (not second)

                    #print(f"createCube:{createCube}")
                    if createCube:
                        #Check if the cuboid has finite volume
                        sizex = ix[1] - ix[0]
                        sizey = iy[1] - iy[0]
                        sizez = iz[1] - iz[0]

                        vol = sizex*sizey*sizez
                        #print(f"new cuboid volume = {vol}")

                        if vol >0:
                            #Create new cuboid using the interval
                            newCuboid = [(ix[0],iy[0],iz[0]) , (ix[1],iy[1],iz[1])]
                            
                            newCuboids.append(newCuboid)
        return newCuboids

    else:
        #No overlap
        if subtract:
            return [c1]
        else:
            return [c1, c2]


In [4]:
def getAddedCuboids(c1,c2):
    #Same as appendCube, but get only a list of added cubes
    newcuboids = appendCube(c1,c2,False)

    #Reject overlaps
    addedCuboids = []
    for c0 in newcuboids:
        if not checkOverlap(c1,c0):
            addedCuboids.append(c0)
    return addedCuboids

In [23]:
def getTotalCubesOn(reactor):
    #get the total volume on cuboids that are on
    totalvol = 0
    for cube0 in reactor:
        xi,yi,zi = cube0[0]
        xf,yf,zf = cube0[1]
        dx = xf-xi
        dy = yf-yi
        dz = zf-zi

        # vol = float(dx)*float(dy)*float(dz)
        vol = dx*dy*dz
        totalvol+=vol
    # return(int(totalvol))
    return totalvol

# Tests

In [28]:
#Crop inside an existing cube cube corner
newcubes=appendCube( [(0,0,0),(3,3,3)] , [(1,1,1),(2,2,2)] , subtract=True)
print(len(newcubes))
newcubes

26


[[(0, 0, 0), (1, 1, 1)],
 [(1, 0, 0), (2, 1, 1)],
 [(2, 0, 0), (3, 1, 1)],
 [(0, 1, 0), (1, 2, 1)],
 [(1, 1, 0), (2, 2, 1)],
 [(2, 1, 0), (3, 2, 1)],
 [(0, 2, 0), (1, 3, 1)],
 [(1, 2, 0), (2, 3, 1)],
 [(2, 2, 0), (3, 3, 1)],
 [(0, 0, 1), (1, 1, 2)],
 [(1, 0, 1), (2, 1, 2)],
 [(2, 0, 1), (3, 1, 2)],
 [(0, 1, 1), (1, 2, 2)],
 [(2, 1, 1), (3, 2, 2)],
 [(0, 2, 1), (1, 3, 2)],
 [(1, 2, 1), (2, 3, 2)],
 [(2, 2, 1), (3, 3, 2)],
 [(0, 0, 2), (1, 1, 3)],
 [(1, 0, 2), (2, 1, 3)],
 [(2, 0, 2), (3, 1, 3)],
 [(0, 1, 2), (1, 2, 3)],
 [(1, 1, 2), (2, 2, 3)],
 [(2, 1, 2), (3, 2, 3)],
 [(0, 2, 2), (1, 3, 3)],
 [(1, 2, 2), (2, 3, 3)],
 [(2, 2, 2), (3, 3, 3)]]

In [29]:
len(newcubes) #Seems to be working

26

In [30]:
#Test Crop on a single axis
newcubes = appendCube( [(0,0,0),(5,1,1)] , [(3,0,0),(10,1,1)] , True)
newcubes

[[(0, 0, 0), (3, 1, 1)]]

In [31]:
#Test subtract on a single axis
newcubes = appendCube( [(0,0,0),(1,5,1)] , [(0,3,0),(1,10,1)] , True)
newcubes

[[(0, 0, 0), (1, 3, 1)]]

In [32]:
#Test add completely seperate cubes
newcubes = appendCube( [(0,0,0),(1,1,1)] , [(5,0,0),(6,1,1)] , False)
newcubes

[[(0, 0, 0), (1, 1, 1)], [(5, 0, 0), (6, 1, 1)]]

In [33]:
#Cut corner add on seperate cubes
newcubes = appendCube( [(0,0,0),(2,2,2)] , [(1,1,1),(4,4,4)] , True)
newcubes

[[(0, 0, 0), (1, 1, 1)],
 [(1, 0, 0), (2, 1, 1)],
 [(0, 1, 0), (1, 2, 1)],
 [(1, 1, 0), (2, 2, 1)],
 [(0, 0, 1), (1, 1, 2)],
 [(1, 0, 1), (2, 1, 2)],
 [(0, 1, 1), (1, 2, 2)]]

In [34]:
len(newcubes)

7

In [35]:
#crop whole cube
newcubes = appendCube( [(0,0,0),(1,1,1)] , [(-10,-10,-10),(4,4,4)] , True)
newcubes

[]

In [36]:
#crop outside the cube
newcubes = appendCube( [(20,20,20),(30,30,30)] , [(-10,-10,-10),(4,4,4)] , True)
print(getTotalCubesOn(newcubes))
newcubes
#OK

1000


[[(20, 20, 20), (30, 30, 30)]]

In [37]:
#Test. crop cube corner
newcubes=appendCube( [(0,0,0),(2,2,2)] , [(-1,-1,-1),(1,1,1)] , subtract=True)
print(getTotalCubesOn(newcubes))
print(len(newcubes))
newcubes

7
7


[[(1, 0, 0), (2, 1, 1)],
 [(0, 1, 0), (1, 2, 1)],
 [(1, 1, 0), (2, 2, 1)],
 [(0, 0, 1), (1, 1, 2)],
 [(1, 0, 1), (2, 1, 2)],
 [(0, 1, 1), (1, 2, 2)],
 [(1, 1, 1), (2, 2, 2)]]

In [38]:
#Test. Append, cube bigger than other
newcubes=appendCube( [(1,1,1),(2,2,2)] , [(0,0,0),(3,3,3)] , subtract=False)
print(len(newcubes))
print(getTotalCubesOn(newcubes))
newcubes

27
27


[[(0, 0, 0), (1, 1, 1)],
 [(1, 0, 0), (2, 1, 1)],
 [(2, 0, 0), (3, 1, 1)],
 [(0, 1, 0), (1, 2, 1)],
 [(1, 1, 0), (2, 2, 1)],
 [(2, 1, 0), (3, 2, 1)],
 [(0, 2, 0), (1, 3, 1)],
 [(1, 2, 0), (2, 3, 1)],
 [(2, 2, 0), (3, 3, 1)],
 [(0, 0, 1), (1, 1, 2)],
 [(1, 0, 1), (2, 1, 2)],
 [(2, 0, 1), (3, 1, 2)],
 [(0, 1, 1), (1, 2, 2)],
 [(1, 1, 1), (2, 2, 2)],
 [(2, 1, 1), (3, 2, 2)],
 [(0, 2, 1), (1, 3, 2)],
 [(1, 2, 1), (2, 3, 2)],
 [(2, 2, 1), (3, 3, 2)],
 [(0, 0, 2), (1, 1, 3)],
 [(1, 0, 2), (2, 1, 3)],
 [(2, 0, 2), (3, 1, 3)],
 [(0, 1, 2), (1, 2, 3)],
 [(1, 1, 2), (2, 2, 3)],
 [(2, 1, 2), (3, 2, 3)],
 [(0, 2, 2), (1, 3, 3)],
 [(1, 2, 2), (2, 3, 3)],
 [(2, 2, 2), (3, 3, 3)]]

In [39]:
#Test. Crop in in a slice fashion, YZ plane
newcubes=appendCube( [(0,0,0),(10,10,10)] , [(5,-100,-100),(100,100,100)] , subtract=True)
print(len(newcubes))
print(getTotalCubesOn(newcubes))
newcubes

1
500


[[(0, 0, 0), (5, 10, 10)]]

In [40]:
#Test. Crop in in a slice fashion
newcubes=appendCube( [(0,0,0),(10,10,10)] , [(-100,5,-100),(100,100,100)] , subtract=True)
print(len(newcubes))
print(getTotalCubesOn(newcubes))
newcubes

1
500


[[(0, 0, 0), (10, 5, 10)]]

In [41]:
#Test. Crop in in a slice fashion
newcubes=appendCube( [(0,0,0),(10,10,10)] , [(-100,-100,5),(100,100,100)] , subtract=True)
print(len(newcubes))
print(getTotalCubesOn(newcubes))
newcubes

1
500


[[(0, 0, 0), (10, 10, 5)]]

In [42]:
#Test. Crop a slice corner
newcubes=appendCube( [(0,0,0),(2,2,2)] , [(1,1,-100),(100,100,100)] , subtract=True)
print(len(newcubes))
print(getTotalCubesOn(newcubes))
newcubes

3
6


[[(0, 0, 0), (1, 1, 2)], [(1, 0, 0), (2, 1, 2)], [(0, 1, 0), (1, 2, 2)]]

In [43]:
#Append cubes, sharing corner
newcubes=appendCube( [(0,0,0),(2,2,2)] , [(1,1,1),(3,3,3)] , subtract=False)
print(len(newcubes))
print(getTotalCubesOn(newcubes))
newcubes

15
15


[[(0, 0, 0), (1, 1, 1)],
 [(1, 0, 0), (2, 1, 1)],
 [(0, 1, 0), (1, 2, 1)],
 [(1, 1, 0), (2, 2, 1)],
 [(0, 0, 1), (1, 1, 2)],
 [(1, 0, 1), (2, 1, 2)],
 [(0, 1, 1), (1, 2, 2)],
 [(1, 1, 1), (2, 2, 2)],
 [(2, 1, 1), (3, 2, 2)],
 [(1, 2, 1), (2, 3, 2)],
 [(2, 2, 1), (3, 3, 2)],
 [(1, 1, 2), (2, 2, 3)],
 [(2, 1, 2), (3, 2, 3)],
 [(1, 2, 2), (2, 3, 3)],
 [(2, 2, 2), (3, 3, 3)]]

In [126]:
#Append cubes, sharing corner

'''
Current reactor
[[(-5812, 98018, 33935), (-1261, 100708, 72968)], [(-53470, -120233, -44150), (21292, -33475, 38148)]]
Total ON cubes in reactor: 534279268349878
NEW cuboid: [(-93533, -16170, -104985), (-4275, 68772, -24506)] , subtract=True
Current reactor
[[(-53470, -120233, -44150), (21292, -33475, 38148)]]
Total ON cubes in reactor: 533801418947608
'''

newcubes=appendCube( [(-5812, 98018, 33935), (-1261, 100708, 72968)] , [(-93533, -16170, -104985), (-4275, 68772, -24506)] , subtract=True)
print(len(newcubes))
print(getTotalCubesOn(newcubes))
newcubes
#Not consistent with the calculation

1
477849402270


[[(-5812, 98018, 33935), (-1261, 100708, 72968)]]

## Run on the data

In [13]:
#filename = "input_test.txt" #Expected ??, with 590784 in initialisation region
#filename = "input_reboot_test.txt" #Expected 2758514936282235 to be on, with 474140 in initialisation region
#filename = "input_test_small2.txt"
#filename = "input_test_small3.txt"
#filename = "input_reboot_test_short.txt" #To compare with numpy, ok! comparing successful

filename = "input.txt" #45591605216010 is wrong, 1267133912086024 ?

In [14]:
with open(filename,'r') as f:
    lines = f.read().splitlines()
#print(lines)

In [15]:
def removeCuboidFromReactor(reactorCuboids, removecuboid):
    reactorCuboids0=[]
    for cub0 in reactorCuboids:
        reactorCuboids0.extend(appendCube(cub0, removecuboid, subtract = True))
    return reactorCuboids0

In [16]:
#Test potential problem
'''
Current reactor
[[(-5812, 98018, 33935), (-1261, 100708, 72968)], [(-53470, -120233, -44150), (21292, -33475, 38148)]]
Total ON cubes in reactor: 534279268349878
NEW cuboid: [(-93533, -16170, -104985), (-4275, 68772, -24506)] , subtract=True
Current reactor
[[(-53470, -120233, -44150), (21292, -33475, 38148)]]
Total ON cubes in reactor: 533801418947608
'''

testcurrreactor = [[(-5812, 98018, 33935), (-1261, 100708, 72968)], [(-53470, -120233, -44150), (21292, -33475, 38148)]]
removecuboid = [(-93533, -16170, -104985), (-4275, 68772, -24506)]
removeCuboidFromReactor(testcurrreactor,removecuboid)
#Now it's ok

[[(-5812, 98018, 33935), (-1261, 100708, 72968)],
 [(-53470, -120233, -44150), (21292, -33475, 38148)]]

In [17]:
def addCuboidToReactor(reactorCuboids, addcuboid):
    newcuboids = [addcuboid] #To propagate the new cuboids created by adding a single one

    for cub0 in reactorCuboids:
        newcuboids0 = []
        for newcub0 in newcuboids:
            newcub1 = getAddedCuboids(cub0,newcub0)
            newcuboids0.extend(newcub1)

        newcuboids= newcuboids0

    #Add the resulting non-overlap newcuboids to the reactor
    reactorCuboids0=reactorCuboids.copy()
    reactorCuboids0.extend(newcuboids)

    return reactorCuboids0

In [18]:
# def addCuboidToReactor2(reactorCuboids, addcuboid):
#     #TODO, not completed
#     #Do the addition and then resolve the overlapping between the cuboids themselves
#     reactorCuboids0=[]
#     for cub0 in reactorCuboids:
#         res = addcuboid(cub0,addcuboid)
#         reactorCuboids0.append(res)

#     hasOverlaps=True

#     while hasOverlaps:
#         #Check and resolve overlaps
#         for icub1 in range(len(reactorCuboids0)-1):
#             cub1 = reactorCuboids0[icub1]
#             for icub2 in range(icub1,len(reactorCuboids0)):
#                 icub2=reactorCuboids0[icub2]
#                 if checkOverlap(icub1,icub2):
#                     overalapRes = appendCube(icub1,icub2)
                    

#     return reactorCuboids0

In [19]:
#Process line-by-line
#Everytime it adds/removes voxels we need to process through the appendCube function
#By comparing with existing cuboids

reactorCuboids = []
line = 0

for l0 in lines:
    subtract = False
    if l0.find("off")>=0:
        subtract = True
    
    #Get coordinates
    xpos = l0.find("x=")
    ypos = l0.find("y=")
    zpos = l0.find("z=")

    xcoords = list(map(int,l0[xpos+2:ypos].replace(',','').split('..')))
    ycoords = list(map(int,l0[ypos+2:zpos].replace(',','').split('..')))
    zcoords = list(map(int,l0[zpos+2:].replace(',','').split('..')))

    # do +1 to the final voxel
    xcoords[1] +=1
    ycoords[1] +=1
    zcoords[1] +=1
    
    if xcoords[0]>xcoords[1] or ycoords[0]>ycoords[1] or zcoords[0]>zcoords[1]:
        print("WARNING, coordinates not ordered.")

    #print(f"NEW xcoords, ycoords, zcoords = {xcoords},{ycoords},{zcoords} ; subtract={subtract}")

    thiscuboid = [ (xcoords[0],ycoords[0],zcoords[0]) , (xcoords[1],ycoords[1],zcoords[1])]
    print(f"Line {line} , NEW cuboid: {thiscuboid} , subtract={subtract}")

    if len(reactorCuboids)==0:
        reactorCuboids.append(thiscuboid)
    else:
        if subtract:
            reactorCuboids = removeCuboidFromReactor(reactorCuboids, thiscuboid)
        else:
            reactorCuboids = addCuboidToReactor(reactorCuboids, thiscuboid)
    
    #print("Current reactor")
    #print(reactorCuboids)
    print(f"len(reactorCuboids) = {len(reactorCuboids)}")
    totalon = getTotalCubesOn(reactorCuboids)
    print(f"Total ON cubes in reactor: {totalon}")
    
    line+=1
    #react0.turnOnOrOff(xcoords[0], xcoords[1] , ycoords[0], ycoords[1] , zcoords[0],zcoords[1], turn)


Line 0 , NEW cuboid: [(-25, -37, -38), (23, 18, 9)] , subtract=False
len(reactorCuboids) = 1
Total ON cubes in reactor: 124080
Line 1 , NEW cuboid: [(-1, -43, -22), (46, 12, 29)] , subtract=False
len(reactorCuboids) = 8
Total ON cubes in reactor: 219459
Line 2 , NEW cuboid: [(-20, -29, -16), (27, 25, 30)] , subtract=False
len(reactorCuboids) = 23
Total ON cubes in reactor: 258622
Line 3 , NEW cuboid: [(-49, -9, -3), (5, 41, 48)] , subtract=False
len(reactorCuboids) = 47
Total ON cubes in reactor: 366652
Line 4 , NEW cuboid: [(-36, -30, -14), (11, 19, 31)] , subtract=False
len(reactorCuboids) = 70
Total ON cubes in reactor: 384030
Line 5 , NEW cuboid: [(-45, -32, -33), (3, 17, 19)] , subtract=False
len(reactorCuboids) = 88
Total ON cubes in reactor: 413261
Line 6 , NEW cuboid: [(-7, -7, -31), (40, 48, 18)] , subtract=False
len(reactorCuboids) = 116
Total ON cubes in reactor: 477316
Line 7 , NEW cuboid: [(-13, -41, -8), (36, 14, 40)] , subtract=False
len(reactorCuboids) = 160
Total ON cu

In [21]:
reactorCuboids

[[(-25, -37, -38), (23, 18, 9)],
 [(-1, -43, -22), (23, -37, 9)],
 [(23, -43, -22), (46, -37, 9)],
 [(23, -37, -22), (46, 12, 9)],
 [(-1, -43, 9), (23, -37, 29)],
 [(23, -43, 9), (46, -37, 29)],
 [(-1, -37, 9), (6, -35, 15)],
 [(6, -37, 9), (23, -35, 15)],
 [(-1, -35, 9), (6, -24, 15)],
 [(6, -35, 9), (23, -24, 15)],
 [(-1, -24, 9), (6, 12, 15)],
 [(6, -24, 9), (23, 12, 15)],
 [(-1, -37, 15), (6, -35, 29)],
 [(6, -37, 15), (23, -35, 29)],
 [(6, -35, 15), (23, -24, 29)],
 [(-1, -24, 15), (6, 12, 29)],
 [(6, -24, 15), (23, 12, 29)],
 [(23, -37, 9), (46, 12, 29)],
 [(23, 12, -16), (27, 18, 9)],
 [(-20, 18, -16), (23, 25, 9)],
 [(23, 18, -16), (27, 25, 9)],
 [(-20, -29, 9), (-11, -24, 15)],
 [(-11, -29, 9), (-1, -24, 15)],
 [(-20, -24, 9), (-11, 12, 15)],
 [(-11, -24, 9), (-1, 12, 15)],
 [(-20, -29, 15), (-11, -24, 29)],
 [(-20, -24, 15), (-11, 12, 29)],
 [(-11, -24, 15), (-1, 12, 29)],
 [(-20, 12, 9), (-1, 18, 29)],
 [(-1, 12, 9), (23, 18, 29)],
 [(-20, -29, 29), (-11, -24, 30)],
 [(-20, 

In [24]:
print(getTotalCubesOn(reactorCuboids)) #OK, 1267133912086024

1267133912086024


### Get the number of cubes ON inside the intialisation region -50 to 50

In [155]:
import numpy as np
class reactor:

    def __init__(self, xmin,xmax , ymin,ymax , zmin, zmax):
        shape=( xmax-xmin+1 , ymax-ymin+1 , zmax-zmin+1)
        self.volume = np.zeros( shape, dtype=np.int32 )
        self.xmin = xmin
        self.xmax = xmax
        self.ymin = ymin
        self.ymax = ymax
        self.zmin = zmin
        self.zmax = zmax

    def orderasc(self,x,y):
        x0=x
        y0=y
        if x0>y0:
            return y0,x0
        return x0,y0


    def turnOnOrOff(self, xmin,xmax, ymin,ymax, zmin,zmax, turn="on"):
        xmin,xmax = self.orderasc(xmin,xmax)
        #print(f"xmin,xmax = {xmin},{xmax}")
        if xmin>= self.xmin and xmin < self.xmax and xmax<=self.xmax:
            xc_min = xmin-self.xmin
            xc_max = xmax - self.xmin+1

            ymin,ymax = self.orderasc(ymin,ymax)
            #print(f"ymin,ymax = {ymin},{ymax}")
            if ymin>= self.ymin and ymin < self.ymax and ymax<=self.ymax:
                yc_min = ymin-self.ymin
                yc_max = ymax - self.ymin+1

                zmin,zmax = self.orderasc(zmin,zmax)
                #print(f"zmin,zmax = {zmin},{zmax}")
                if zmin>= self.zmin and zmin < self.zmax and zmax<=self.zmax:
                    zc_min = zmin-self.zmin
                    zc_max = zmax - self.zmin+1

                    #modshape = (xc_max-xc_min, yc_max-yc_min , zc_max-zc_min)
                    turnval = 0
                    if turn=="on":
                        turnval=1
                    
                    vol = (zc_max-zc_min)*(yc_max-yc_min)*(xc_max-xc_min)

                    #print(f"xc : {xc_min},{xc_max} ; yc: {yc_min},{yc_max} ; zc: {zc_min},{zc_max} ; volume = {vol}, turnval={turnval}")

                    self.volume[xc_min:xc_max, yc_min:yc_max , zc_min:zc_max] = turnval
                else:
                    print("z out of bounds")
                    return
            else:
                print("y out of bounds")
                return
        else:
            print("x out of bounds")
            return

In [156]:
#Get total cubes in initialisation region
react0 = reactor(-50,50, -50,50 , -50,50)
for c0 in reactorCuboids:
    react0.turnOnOrOff(c0[0][0] , c0[1][0] , c0[0][1] , c0[1][1] , c0[0][2] , c0[1][2] )
np.sum(react0.volume)

x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of bounds
x out of

495082